# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 3:14PM,255149,19,ADV80010457,"AdvaGen Pharma, Ltd",Released
1,Jan 16 2023 3:07PM,255148,19,9039124,VITABIOTICS USA INC.,Released
2,Jan 16 2023 2:45PM,255142,10,MSP217590,"Methapharm, Inc.",Released
3,Jan 16 2023 2:45PM,255142,10,MSP217591,"Methapharm, Inc.",Released
4,Jan 16 2023 2:45PM,255142,10,MSP217592,"Methapharm, Inc.",Released
5,Jan 16 2023 2:45PM,255142,10,MSP217594,"Methapharm, Inc.",Released
6,Jan 16 2023 2:45PM,255142,10,MSP217595,"Methapharm, Inc.",Released
7,Jan 16 2023 2:45PM,255142,10,MSP217596,"Methapharm, Inc.",Released
8,Jan 16 2023 2:45PM,255142,10,MSP217597,"Methapharm, Inc.",Released
9,Jan 16 2023 2:45PM,255142,10,MSP217598,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,255145,Executing,2
28,255145,Released,4
29,255146,Released,1
30,255148,Released,1
31,255149,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255143,NaN,NaN,14.0
255145,NaN,2.0,4.0
255146,NaN,NaN,1.0
255148,NaN,NaN,1.0
255149,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255006,50.0,1.0,1.0
255049,0.0,0.0,42.0
255053,0.0,0.0,1.0
255087,0.0,1.0,0.0
255090,6.0,13.0,29.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255006,50,1,1
255049,0,0,42
255053,0,0,1
255087,0,1,0
255090,6,13,29


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255006,50,1,1
1,255049,0,0,42
2,255053,0,0,1
3,255087,0,1,0
4,255090,6,13,29


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255006,50,1,1
1,255049,,,42
2,255053,,,1
3,255087,,1,
4,255090,6,13,29


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 3:14PM,255149,19,"AdvaGen Pharma, Ltd"
1,Jan 16 2023 3:07PM,255148,19,VITABIOTICS USA INC.
2,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc."
20,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.
26,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation
40,Jan 16 2023 2:44PM,255146,10,Methapharm-G
41,Jan 16 2023 2:38PM,255141,10,"Snap Medical Industries, LLC"
44,Jan 16 2023 2:36PM,255139,10,ISDIN Corporation
47,Jan 16 2023 2:15PM,255136,10,"Citieffe, Inc."
49,Jan 16 2023 2:10PM,255135,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 16 2023 3:14PM,255149,19,"AdvaGen Pharma, Ltd",,,1
1,Jan 16 2023 3:07PM,255148,19,VITABIOTICS USA INC.,,,1
2,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",,,18
3,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,,2,4
4,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,,,14
5,Jan 16 2023 2:44PM,255146,10,Methapharm-G,,,1
6,Jan 16 2023 2:38PM,255141,10,"Snap Medical Industries, LLC",,,3
7,Jan 16 2023 2:36PM,255139,10,ISDIN Corporation,,,3
8,Jan 16 2023 2:15PM,255136,10,"Citieffe, Inc.",,,2
9,Jan 16 2023 2:10PM,255135,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 3:14PM,255149,19,"AdvaGen Pharma, Ltd",1,,
1,Jan 16 2023 3:07PM,255148,19,VITABIOTICS USA INC.,1,,
2,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",18,,
3,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,4,2,
4,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,14,,
5,Jan 16 2023 2:44PM,255146,10,Methapharm-G,1,,
6,Jan 16 2023 2:38PM,255141,10,"Snap Medical Industries, LLC",3,,
7,Jan 16 2023 2:36PM,255139,10,ISDIN Corporation,3,,
8,Jan 16 2023 2:15PM,255136,10,"Citieffe, Inc.",2,,
9,Jan 16 2023 2:10PM,255135,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 3:14PM,255149,19,"AdvaGen Pharma, Ltd",1,,
1,Jan 16 2023 3:07PM,255148,19,VITABIOTICS USA INC.,1,,
2,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",18,,
3,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,4,2,
4,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 3:14PM,255149,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jan 16 2023 3:07PM,255148,19,VITABIOTICS USA INC.,1.0,NaN,NaN
2,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",18.0,NaN,NaN
3,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,4.0,2.0,NaN
4,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 3:14PM,255149,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jan 16 2023 3:07PM,255148,19,VITABIOTICS USA INC.,1.0,0.0,0.0
2,Jan 16 2023 2:45PM,255142,10,"Methapharm, Inc.",18.0,0.0,0.0
3,Jan 16 2023 2:44PM,255145,19,VITABIOTICS USA INC.,4.0,2.0,0.0
4,Jan 16 2023 2:44PM,255143,10,ISDIN Corporation,14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3316651,95.0,0.0,0.0
12,255115,1.0,0.0,0.0
15,510055,43.0,1.0,50.0
19,2041029,38.0,16.0,6.0
20,255123,0.0,1.0,0.0
21,255087,0.0,1.0,0.0
22,510149,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3316651,95.0,0.0,0.0
1,12,255115,1.0,0.0,0.0
2,15,510055,43.0,1.0,50.0
3,19,2041029,38.0,16.0,6.0
4,20,255123,0.0,1.0,0.0
5,21,255087,0.0,1.0,0.0
6,22,510149,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,95.0,0.0,0.0
1,12,1.0,0.0,0.0
2,15,43.0,1.0,50.0
3,19,38.0,16.0,6.0
4,20,0.0,1.0,0.0
5,21,0.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,95.0
1,12,Released,1.0
2,15,Released,43.0
3,19,Released,38.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,50.0,6.0,0.0,0.0,0.0
Executing,0.0,0.0,1.0,16.0,1.0,1.0,0.0
Released,95.0,1.0,43.0,38.0,0.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,50.0,6.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,16.0,1.0,1.0,0.0
2,Released,95.0,1.0,43.0,38.0,0.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,50.0,6.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,16.0,1.0,1.0,0.0
2,Released,95.0,1.0,43.0,38.0,0.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()